### Model Ensembling

**This is a sample of how to use following Model_Ensembling classes**

convert a file into a dictionary respresenting results from image captioning based model

In [141]:
def cal_caption_result (caption_sim_wmd_file):
    lines_caption_sim = []
    with open(caption_sim_wmd_file) as f:
        lines_caption_sim = f.readlines()
    ar_cap_sim_dic={}
    for line in lines_caption_sim:
        segs=line.strip().split("\t")
        if segs[0] not in ar_cap_sim_dic:
            ar_cap_sim_dic[segs[0]]={}
        ar_cap_sim_dic[segs[0]][os.path.splitext(segs[1])[0]]= 1-float(segs[2])
    return ar_cap_sim_dic

In [142]:
ar_cap_sim_dic=cal_caption_result(r"result\test_caption_sim_wmd.txt")

In [143]:
print(len(ar_cap_sim_dic))

1915


convert a file into a dictionary respresenting results from face name matching based model

In [144]:
def face_matching_result (face_matching_file):
    image_train_sim = []
    with open(face_matching_file) as f:
        image_train_sim = f.readlines()
    ar_train_sim_dic={}
    for line in image_train_sim:
        segs=line.strip().split("\t")
        if segs[0] not in ar_train_sim_dic:
            ar_train_sim_dic[segs[0]]=[]
        ar_train_sim_dic[segs[0]].append((os.path.splitext(segs[1])[0], segs[2], segs[3]))
    ar_train_sim_dic_cal={}
    
    for k, v in ar_train_sim_dic.items():
        if k not in ar_train_sim_dic_cal:
            ar_train_sim_dic_cal[k]={}
        for item in v:
            if int(item[2])==0:
                sim=0
            else:
                sim=(1-float(item[1]))*int(item[2])
            ar_train_sim_dic_cal[k][item[0]]= sim
    return ar_train_sim_dic_cal

In [145]:
ar_train_sim_dic_cal=face_matching_result(r"result\test_train_img_similarity.txt")

In [146]:
ar_crawl_sim_dic_cal=face_matching_result(r"result\test_crawl_img_similarity.txt")

In [147]:
print(len(ar_crawl_sim_dic_cal))

191


normalize value of a given dictionary (Min-max normalization)

In [148]:
def norm_dict(a_dict):
    result={}
    amin, amax = min(a_dict.values()), max(a_dict.values())
    for k, v in a_dict.items():
        if amax-amin==0:
            result[k]=0
        else:
            result[k] = (v-amin) / (amax-amin)
    return result

normalize value of a given dictionary 
the dictionary respresents results from face name matching based model

In [149]:
def norm_sim(a_dict):
    result={}
    print(len(a_dict))
    for k, v in a_dict.items():
        result[k]=norm_dict(v)
    return result

sort the value in the given dictionary

In [150]:
def sort_dict(a_dict):
    normalized_dict=norm_sim(a_dict)
    result = {}
    for k, v in normalized_dict.items():
        sort_v = dict(sorted(v.items(), key=lambda item: item[1], reverse=True))
        result[k] = sort_v
    return result

sort dictionary respresenting results from face name matching based model
sort dictionary respresenting results from image captioning matching based model

In [151]:
cap_dict=sort_dict(ar_cap_sim_dic)
train_dict=sort_dict(ar_train_sim_dic_cal)
crawl_dict=sort_dict(ar_crawl_sim_dic_cal)

1915
128
191


merge results from image captioning based model and results from face matching based model

In [152]:
def merge_cap_face(cap_dict, train_dict, crawl_dict, weight_cap, weight_img):
    result={}
    for k, v in cap_dict.items():
        img_id=os.path.splitext(k)[0]
        result[img_id]=v
        if k in train_dict:
            for k_tr, v_tr in train_dict[k].items():
                result[img_id][k_tr]=v_tr*weight_img
        if k in crawl_dict:
            for k_cr, v_cr in crawl_dict[k].items():
                result[img_id][k_cr]=v_cr*weight_img
    return result

In [153]:
cap_face_result=merge_cap_face(cap_dict, train_dict, crawl_dict, 0.5, 0.5)

In [155]:
sorted_cap_face_result=sort_dict(cap_face_result)

1915


truncate image candidates into top 100 list for each article

In [156]:
def truncate_result(a_dict):
    for key, value in a_dict.items():
        l = [*value]
        l_100 = l[0:100]
        a_dict[key]=l_100

In [157]:
truncate_result(sorted_cap_face_result)

In [158]:
len(sorted_cap_face_result)

1915

acquire result from url matching based method

In [159]:
img_id_name_dict = extract_img_url_token("../data/MediaEvalNewsImagesBatch04images.tsv",
                                              TEST_I_ID_IDX,
                                              TEST_IMG_URL_IDX)
article_id_name_dict = extract_article_token("../data/MediaEvalNewsImagesBatch04articles.tsv", 
                                             TEST_A_ID_IDX,
                                             TEST_TITLE_IDX)
url_result = match_url(article_id_name_dict, img_id_name_dict)

matching url
1772
1772


merge result from url matching based model into result

In [160]:
def merge_url(final_cap_face_result, url_result):
    final_result={}
    for s_id, s_value in final_cap_face_result.items():
        if s_id in result:
            diff_elements = [x for x in url_result[s_id] if x not in s_value ]
            common_elements= [x for x in s_value if x in url_result[s_id] ]
            tail_elements=s_value[len(s_value)-len(diff_elements):]
            common_ele_in_tail=[x for x in common_elements if x in tail_elements]
            if len(common_ele_in_tail)>0:
                new_value=diff_elements+common_ele_in_tail+s_value[:len(s_value)-len(diff_elements)-len(common_ele_in_tail)]
            else:
                new_value=diff_elements+s_value[:len(s_value)-len(diff_elements)]
            final_result[s_id]= new_value

        else:
            final_result[s_id]=s_value
    return final_result

In [161]:
final_result=merge_url(sorted_cap_face_result, url_result)

save final result into a file

In [162]:
def save_final_result(output_file, final_result):
    with open(output_file, "w") as the_file:
        header="particleID"
        for i in range(100):
            header+="\t"+"iid"+str(i+1)
        the_file.write(header+"\n")
        for key, value in final_result.items():
            the_file.write(key+'\t'+ "\t".join(value)+"\n")

In [163]:
save_final_result(r"result\final_result.tsv", final_result)

Result Exploring

In [165]:
df=pd.read_csv(r"result\final_result.tsv", delimiter="\t")

In [166]:
df

,particleID,iid1,iid2,iid3,iid4,iid5,iid6,iid7,iid8,iid9,...,iid91,iid92,iid93,iid94,iid95,iid96,iid97,iid98,iid99,iid100
0,1000265260,134746,134710,135977,134622,134853,136039,136139,136193,134762,...,134315,134322,134510,135693,134997,135109,135698,134633,136319,134726
1,1001935289,135908,135628,134775,136331,134390,136306,134381,134409,134909,...,136045,136458,136137,134266,134868,135007,135435,136062,136172,136179
2,1002375244,136277,134791,136639,134624,134416,135315,135770,136169,136530,...,135374,135406,135453,135626,135924,135952,136016,136106,136168,136189
3,1002735288,136819,136242,134962,134606,136361,134390,134123,134231,134332,...,136561,134322,136431,134502,134709,136243,134149,134162,134733,134788
4,1002835245,135405,136751,134806,135863,136083,135390,136320,136390,134853,...,135534,135709,134206,136624,136931,136639,136288,134634,134627,135606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,1999075246,135716,134131,135146,135009,135268,135742,135805,136327,134737,...,137028,135043,134556,134699,134729,134770,134923,135112,135368,135506
1911,1999165241,134482,135136,135987,136178,136355,136646,136889,136763,134782,...,136791,135627,136258,134220,134356,136583,136668,134291,136412,136326
1912,1999345240,136293,135876,135329,136288,136356,135714,134205,134395,135003,...,136679,136669,134933,136116,135566,136096,135390,135844,136285,134557
1913,1999355239,134193,136200,135183,135391,135705,135853,135432,135722,135813,...,136327,134336,134523,136558,136988,134946,135912,136476,134380,134397
